<table width="100%">
  <tr>
    <td width="50%" align="left">
      <img src="imgs/Brockmann_Consult_Logo_Horizontal_Tagline_1.1.png" alt="Company Logo" style="max-width: 100%; height: auto;" width="300">
    </td>
    <td width="50%" align="right">
      <img src="imgs/logo-light.png" alt="xcube logo" style="max-width: 100%; height: auto;" width="150">
    </td>
  </tr>
</table>

<table width="100%">
  <tr>
    <td width="100%" align="center">
      <h1 style="text-align: center; margin-bottom: 0.3em; font-weight: bold;">
        Building Sentinel-2 Data Cubes
      </h1>
      <div style="height: 0.5em;"></div>
      <h2 style="text-align: center; margin-top: 0; font-weight: bold;">
        Using xcube-stac
      </h2>
    </td>
  </tr>
</table>


**Author(s):** Clara Backens, Konstantin Ntokas, Pontus Lurcock  
**Affiliation:** Brockmann Consult GmbH  
**GitHub:** [xcube-dev/xcube](https://github.com/xcube-dev/xcube), [xcube-dev/xcube-stac](https://github.com/xcube-dev/xcube-stac)   
**Documentation:** [xcube](https://xcube.readthedocs.io/en/latest/), [xcube-stac](https://github.com/xcube-dev/xcube-stac/blob/main/README.md)

---

## Introduction

`xcube-stac` is a Python package and [xcube plugin](https://xcube.readthedocs.io/en/latest/plugins.html) that adds [data stores](https://xcube.readthedocs.io/en/latest/api.html#data-store-framework) which allow to retrieve data organized by a STAC Catalog.


### General structure of a STAC catalog

A SpatioTemporal Asset Catalog (STAC) is built from three main components:

- **Catalogs**
- **Collections**
- **Items**

An **item** represents a single observation and includes:

- a timestamp or time range  
- a spatial bounding box  
- one or more assets (e.g., image bands, metadata)

Items within a **collection** share common characteristics (e.g., satellite observation from one Sentinel mission), and each item corresponds to one measurement at a specific time and place. STAC catalogs can be queried by **spatial and temporal extent**, allowing to efficiently retrieve only the data relevant to their area and time of interest.


<div style="border: 2.5px solid #999; padding: 1em; border-radius: 6px; background-color: #eaeaea;">
  <strong>Objectives:</strong>
  <ul>
    <li>Get an overview what xcube-stac supports so far.</li>
    <li>Learn how to use xcube-stac to open multiple Sentinel-2 observations as analysis-ready datacubes (ARDC).</li>
    <li>Understand what the algorithm does to generate the ARDC.</li>
  </ul>
</div>

> **Disclaimer**  
> This notebook demonstrates the use of open source software and is intended for educational and illustrative purposes only. All software used is subject to its respective licenses. The authors and contributors of this notebook make no guarantees about the accuracy, reliability, or suitability of the content or included code. Use at your own discretion and risk. No warranties are provided, either express or implied.


---

## Install the xcube-stac Data Store

The xcube-stac package is implemented as a **xcube plugin** and can be installed using `conda/mamba` from the conda-forge channel.

- 🐍 **Conda (conda-forge):** [xcube-stac on Anaconda](https://anaconda.org/conda-forge/xcube-stac) `conda install -c conda-forge xcube-stac`

  You can also use **Mamba** as a faster alternative to Conda: `mamba install -c conda-forge xcube-stac`

---

## Section 1

Add descriptive text cells to explain the code and provide background information about the software shown. Ensure the notebook is clearly structured and filled with informative commentary to enhance understanding.

For an example of well-documented content, see:  
[Introduction to the xarray-eopf plugin](https://eopf-sample-service.github.io/eopf-sample-notebooks/introduction-xarray-eopf-plugin)


---

In [1]:
import dask
import xarray as xr
from xcube_multistore import MultiSourceDataStore
from xcube.webapi.viewer import Viewer

You can explore the configuration schema using the method `get_config_schema()`. Run it and expand the fields to learn about the available properties and options.  

For additional guidance, refer to:  
- [Configuration Guide](https://xcube-dev.github.io/xcube-multistore/config/)  
- Notebook: [Setup Config YAML File](https://xcube-dev.github.io/xcube-multistore/examples/setup_config/)  

These resources provide detailed instructions for setting up configuration files.

In [2]:
MultiSourceDataStore.get_config_schema()

In the next cells, we create the configuration dictionary. Since the data sources have different temporal resolutions, some preporcessing is applied to combine them together to a uniform datacube:

| Dataset    | native time steps   | final time steps                 |
|------------|---------------------|----------------------------------|
| Sentinel-2 | irregular time axis | daily (if oberservation exsists) |
| Sentinel-3 | irregular time axis | daily (if oberservation exsists) |
| ERA5-Land  | hourly              | aggregated to daily              |


First, we define common temporal and spatial ranges. In this example, we select the region around Mount Etna for June 2025.

In [3]:
time_range = ["2025-06-10", "2025-06-15"]
bbox = [14.8, 37.45, 15.3, 37.85]

First, we create the configuration for Sentinel-2 where we retrieve the red (`b04`), green (`b03`) and blue (`b02`) band and the scene classification layer.

In [4]:
config_sen2_variable = dict(  
    identifier="sen2_l2a",
    store="eopf-zarr",
    data_id="sentinel-2-l2a",
    open_params=dict(
        time_range=time_range,
        bbox=bbox,
        spatial_res=10 / 111320, # meters converted to degrees (approx.)
        crs="EPSG:4326",
        variables=["b02", "b03", "b04", "scl"],
    ),
    temporal_resample_params=dict(
        frequency="1D",
        agg_methods="mean",
    )
)

Second, we create the configuration for Sentinel-3 where we retrieve the red (`oa09_radiance`), green (`oa06_radiance`) and blue (`oa04_radiance`) band.

In [5]:
config_sen3_variable = dict(
    identifier="sen3_olci",
    store="eopf-zarr",
    data_id="sentinel-3-olci-l1-efr",
    open_params=dict(
        time_range=time_range,
        bbox=bbox,
        spatial_res=10 / 111320, # meters converted to degrees (approx.)
        crs="EPSG:4326",
        variables=["oa04_radiance", "oa06_radiance", "oa09_radiance"],
    ),
    temporal_resample_params=dict(
        frequency="1D",
        agg_methods="mean",
    )
)

Third, we create the configuration for ERA5-land.

In [6]:
config_era5_variable = dict(
    identifier="era5",
    store="cds",
    data_id="reanalysis-era5-single-levels:reanalysis",
    open_params=dict(
        variable_names=[
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "2m_temperature",
            "sea_surface_temperature",
            "total_precipitation",
        ],
        time_range=time_range,
        bbox=[bbox[0] - 0.5, bbox[1] - 0.5, bbox[2] + 0.5, bbox[3] + 0.5],
    ),
    temporal_resample_params=dict(
        frequency="1D",
        agg_methods=dict(
            era5_t2m="mean",
            era5_sst="mean",
            era5_u10="mean",
            era5_v10="mean",
            era5_tp="sum",
        )
    )
)

Now we can combine all dataset variables into one unified datacube. 

In [7]:
config = dict(
    datasets=[
        dict(
            identifier="final_datacube",
            variables=[
                config_sen2_variable,
                config_sen3_variable,
                config_era5_variable,
            ],
        )
    ]
)

Next, we define the required data stores:  

- **storage**: defines where the final data will be written  
- **eopf-zarr**: for accessing Sentinel-2 data  
- **cds**: for accessing ERA5 data 

In [8]:
config["data_stores"] = [
    dict(
        identifier="storage",
        store_id="file",
        store_params=dict(root="data"),
    ),
    dict(
        identifier="eopf-zarr",
        store_id="eopf-zarr",
    ),
    dict(
        identifier="cds",
        store_id="cds",
        store_params=dict(
            endpoint_url="https://cds.climate.copernicus.eu/api",
            cds_api_key="8ba066d4-5195-4c1d-af91-6fb13bfc22df",
            normalize_names=True,
        )
    ),
]

Next we initialize the Mulit-Source Data Store with the configuration. 

In [9]:
msds = MultiSourceDataStore(config)

To inspect the configuration, use the `display_config` method by running the following cell.

In [10]:
msds.display_config()

User-defined ID,Data Store ID,Data Store Params,Data ID,Open Data Params,Grid-Mapping,Format
final_datacube,eopf-zarr,-,sentinel-2-l2a,"time_range: ['2025-06-10', '2025-06-15']; bbox: [14.8, 37.45, 15.3, 37.85]; spatial_res: 8.983111749910169e-05; crs: EPSG:4326; variables: ['b02', 'b03', 'b04', 'scl']",-,Zarr
final_datacube,eopf-zarr,-,sentinel-3-olci-l1-efr,"time_range: ['2025-06-10', '2025-06-15']; bbox: [14.8, 37.45, 15.3, 37.85]; spatial_res: 8.983111749910169e-05; crs: EPSG:4326; variables: ['oa04_radiance', 'oa06_radiance', 'oa09_radiance']",-,Zarr
final_datacube,cds,endpoint_url: https://cds.climate.copernicus.eu/api; cds_api_key: 8ba066d4-5195-4c1d-af91-6fb13bfc22df; normalize_names: True,reanalysis-era5-single-levels:reanalysis,"variable_names: ['10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature', 'sea_surface_temperature', 'total_precipitation']; time_range: ['2025-06-10', '2025-06-15']; bbox: [14.3, 36.95, 15.8, 38.35]",-,Zarr


We can inspect the location of the final datastore using the method `display_geolocations`.

In [11]:
msds.display_geolocations()

Now, we can create a `MultiSourceDataStore` and pass the configuration dictionary, which will initiate the cube generation process. For more robust performance, we limit the Dask scheduler to 4 workers.

> 💡 **Note:**  
> The configuration can also be saved to a YAML file for persistence. The store accepts either a file path to the YAML configuration or the configuration dictionary directly.

In [12]:
dask.config.set(scheduler="threads", num_workers=4)
msds.generate()

Dataset identifier,Status,Message,Exception
final_datacube,COMPLETED,Dataset 'final_datacube' finished: 0:12:03,-


xcube-cds version 1.1.0.dev0
2025-11-25 16:46:05,493 INFO Request ID is 4609a197-b725-4aad-8548-7ba2b8850988
2025-11-25 16:46:05,586 INFO status has been updated to accepted
2025-11-25 16:46:14,312 INFO status has been updated to running
2025-11-25 16:46:19,616 INFO status has been updated to successful


1788648f464cc921bd0459b1995bde1a.zip:   0%|          | 0.00/128k [00:00<?, ?B/s]

/home/konstantin/micromamba/envs/xcube-multistore/lib/python3.13/site-packages/xcube_cds/datasets/reanalysis_era5.py:349: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds = xr.open_mfdataset(file_paths, engine="netcdf4")
/home/konstantin/micromamba/envs/xcube-multistore/lib/python3.13/site-packages/xcube_cds/datasets/reanalysis_era5.py:349: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_comb

We can now open the final data cube using the `storage` data store, as demonstrated below.

In [13]:
ds = msds.stores.storage.open_data("final_datacube.zarr")
ds

<xarray.Dataset> Size: 10GB
Dimensions:                       (time: 6, lat: 4453, lon: 5566)
Coordinates:
  * lat                           (lat) float64 36kB 37.85 37.85 ... 37.45 37.45
  * lon                           (lon) float64 45kB 14.8 14.8 ... 15.3 15.3
    number                        int64 8B ...
    spatial_ref                   int64 8B ...
  * time                          (time) datetime64[ns] 48B 2025-06-10 ... 20...
Data variables:
    era5_sst_mean                 (time, lat, lon) float32 595MB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    era5_t2m_mean                 (time, lat, lon) float32 595MB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    era5_tp_sum                   (time, lat, lon) float32 595MB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    era5_u10_mean                 (time, lat, lon) float32 595MB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    era5_v10_mean                 (time, lat, lon) float32 595MB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    sen2_l2a_b02_mean             (time, lat, lon) float64 1GB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    sen2_l2a_b03_mean             (time, lat, lon) float64 1GB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    sen2_l2a_b04_mean             (time, lat, lon) float64 1GB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    sen2_l2a_scl_mean             (time, lat, lon) float64 1GB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    sen3_olci_oa04_radiance_mean  (time, lat, lon) float32 595MB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    sen3_olci_oa06_radiance_mean  (time, lat, lon) float32 595MB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
    sen3_olci_oa09_radiance_mean  (time, lat, lon) float32 595MB dask.array<chunksize=(1, 1830, 1830), meta=np.ndarray>
Attributes: (16)

We can now view the final data cube in xcube Viewer.

In [14]:
viewer = Viewer()
viewer.add_dataset(ds)
viewer.show()

404 GET /viewer/config/config.json (127.0.0.1): xcube viewer has not been been configured
404 GET /viewer/config/config.json (127.0.0.1) 8.52ms
501 GET /viewer/state?key=sentinel (127.0.0.1) 0.83ms
404 GET /viewer/ext/contributions (127.0.0.1) 191.78ms


In [15]:
viewer.info()

Server: http://localhost:8000
Viewer: http://localhost:8000/viewer/?serverUrl=http://localhost:8000


<table width="100%">
  <tr>
    </td>
    <td width="20%"></td>
    <td width="60%" align="center" style="font-size: 0.9em; color: #666; vertical-align: middle;">
      &copy; 2025 Brockmann Consult GmbH &nbsp;|&nbsp;
      <a href="https://www.brockmann-consult.de" target="_blank">Website</a>
    <td width="20%" align="right" style="vertical-align: middle;">
      <img src="Brockmann_Consult_Logo_Horizontal_1.1.png" alt="Company Logo" width="150">
    </td>
  </tr>
</table>